In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df = pd.read_json('data_save.txt',lines=True,dtype={'E':'datetime64[ms]'})

In [3]:
du = df.query('e=="depthUpdate"').dropna(axis=1)
trade = df.query('e!="depthUpdate"').dropna(axis=1)
del df

In [4]:
trade['E'].agg(['min','max']).diff().iloc[-1]

Timedelta('0 days 16:24:39.354000')

In [5]:
n_ticks= 500
trade[f'ticks_{n_ticks}'] = np.arange(0,trade.shape[0]) // n_ticks
du[f'ticks_{n_ticks}'] = np.arange(0,du.shape[0]) // n_ticks

In [6]:
n_ticks= 10
trade[f'ticks_{n_ticks}'] = np.arange(0,trade.shape[0]) // n_ticks
du[f'ticks_{n_ticks}'] = np.arange(0,du.shape[0]) // n_ticks

1)

In [7]:
px.line(trade.groupby(pd.Grouper(key='E',freq='10S'))['t'].nunique(),title='Индикатор скорости сделок за 10 секунд').write_html('plots/1_speed.html')

2)

In [8]:
trade['type'] = trade['p'].diff().where(lambda x: x!=0).fillna(method='ffill').mul(100).clip(-1,1).map({-1:'sell',1:'buy'})

In [9]:
px.line(trade.groupby([pd.Grouper(key='E',freq='10S')])['type'].value_counts().unstack(),title='Индикатор интенсивности продаж/покупок').write_html('plots/2_buysell_value_count.html')

3)

In [10]:
du['b_first_q']=du['b'].explode().dropna().apply(lambda x: x[1] if x[1]!='0.00000000' else np.nan).groupby(level=0).first().astype(float)
du['a_first_q']=du['a'].explode().dropna().apply(lambda x: x[1] if x[1]!='0.00000000' else np.nan).groupby(level=0).first().astype(float)

du['b_first_p']=du['b'].explode().dropna().apply(lambda x: x[0] if x[1]!='0.00000000' else np.nan).groupby(level=0).first().astype(float)
du['a_first_p']=du['a'].explode().dropna().apply(lambda x: x[0] if x[1]!='0.00000000' else np.nan).groupby(level=0).first().astype(float)

In [11]:
du['diff_p']=du[['b_first_p','a_first_p']].diff(axis=1).iloc[:,-1]
du['diff_q']=du[['b_first_q','a_first_q']].diff(axis=1).iloc[:,-1]

In [12]:
px.line(du.groupby(pd.Grouper(key='E',freq='10S'))['diff_p'].mean(),title='Индикатор спреда').write_html('plots/3_spread.html')

In [13]:
px.line(du.groupby(pd.Grouper(key='E',freq='10S'))['b_first_p','a_first_p'].mean()\
    .join(trade.groupby(pd.Grouper(key='E',freq='10S'))['p'].last())\
        ,title='Линии поведение бида и аска относительно цены последней сделки').write_html('plots/3_bidask_price_comp.html')

In [14]:
olhc = trade.groupby('ticks_10')['p'].agg(['first','last','min','max'])
olhc_500 = trade.groupby('ticks_500')['p'].agg(['first','last','min','max'])

In [20]:
fig = go.Figure(data=go.Ohlc(x=olhc.index,open=olhc['first'],close=olhc['last'],low=olhc['min'],high=olhc['max']))
fig.write_html('plots/5_ohlc.html')

In [21]:
fig = go.Figure(data=go.Ohlc(x=olhc_500.index,open=olhc_500['first'],close=olhc_500['last'],low=olhc_500['min'],high=olhc_500['max']))
fig.write_html('plots/5_ohlc_500.html')